In [229]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = set()
    shortest = 99999
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if len(sup) < shortest:
            shortest = len(sup)
            shortest_sup = set([sup])  # found shorter superstring
        elif len(sup) == shortest:
            shortest_sup.add(sup)
    return shortest_sup  # return shortest

In [230]:
scs(['ABC', 'BCA', 'CAB'])

{'ABCAB', 'BCABC', 'CABCA'}

In [231]:
scs(['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'])

{'CCTTGGATTGC', 'GATTGCCTTGG', 'TGCCTTGGATT', 'TGGATTGCCTT'}

In [232]:
def charToQual(c):
    return ord(c) - 33

def loadFastq(filename):
    reads = []
    quals = []
    with open(filename) as f:
        while True:
            id_line = f.readline().rstrip()
            if not id_line:
                break
            assert id_line.startswith('@')
            reads.append(f.readline().rstrip())
            assert f.readline().rstrip() == '+'
            quals.append(map(charToQual, f.readline().rstrip()))
            assert len(reads[-1]) == len(quals[-1])
    return reads, quals

In [233]:
reads = loadFastq('ads1_week4_reads.fq')[0]

In [234]:
len(reads)

1881

In [235]:
len(reads[0])

100

In [236]:
len(reads)

1881

In [243]:
from collections import defaultdict

def buildKmerMap(reads, n):
    m = defaultdict(set)
    for read in reads:
        for i in range(len(read) - n + 1):
            kmer = read[i:i+n]
            m[kmer].add(read)
    return m

def buildOverlapGraph(reads, n):
    kmerMap = buildKmerMap(reads, n)
    overlaps = defaultdict(list)
    for read in reads:
        suffix = read[-n:]
        candidates = kmerMap[suffix]
        for other_read in candidates:
            if other_read == read:
                continue
            overlap_len = overlap(read, other_read, n)
            if overlap_len > 0:
                overlaps[read].append((overlap_len, other_read))
    for read in overlaps.keys():
        overlaps[read].sort()
    return overlaps


def assemble_greedy(reads, n=30):
    # SLOW!!!
    def max_overlap(reads, n):
        ma=None; mb=None; molen=None;
        for a,b in itertools.permutations(reads,2):
            olen = overlap(a,b,n)
            if olen > 0:
                ma,mb,molen = a,b,olen
        return ma,mb,molen

    a,b,olen = max_overlap(reads, n)
    while olen:
        print len(reads)
        reads.remove(a)
        reads.remove(b)
        reads.append(a + b[olen:])
        a,b,olen = max_overlap(reads, n)

    return ''.join(reads)

def assemble_greedy_2(reads, n=30):
    olg = buildOverlapGraph(reads, n)
    reads = set(reads)
    
    # build a list of all overlaps, sorted by overlap length
    overlaps = []
    for (a,olaps) in olg.iteritems():
        for (olen,b) in olaps:
            overlaps.append((olen,a,b))
    overlaps.sort()

    def rebuild_overlaps(overlaps,a,b,c):
        # rebuild the list of overlaps such that a and b have been 
        # used (ans so cannot be used again), and replaced with the 
        # single read that is the overlap of a and b.
        def olps(overlaps):
            for (o,p,q) in overlaps:
                if p == a or q == b:
                    continue
                elif q == a:
                    yield (o,p,c)
                elif p == b:
                    yield (o,c,q)
                else:
                    yield (o,p,q)
        new_overlaps = list(olps(overlaps))
        new_overlaps.sort()
        return new_overlaps
    
    while overlaps:
        olen,a,b = overlaps.pop()
        merged = a + b[olen:]
        overlaps = rebuild_overlaps(overlaps, a, b, merged)
        reads.remove(a)
        reads.remove(b)
        reads.add(merged)
        
    return ''.join(reads)
    
        

In [244]:
print assemble_greedy_2(['abc', 'zab', 'cde'], 1)

zabcde


In [245]:
import time
start = time.clock()
vg = assemble_greedy_2(reads, 30)
print time.clock() - start

3.22479


In [246]:
len(vg)

15894

In [247]:
vg.count('A')

4633

In [248]:
vg.count('T')

3723